This executive notebook contains the architecture for both image recognition algorithms. The ACDA algorithm does not contain the adaptive kernel weight csv output as it was not deemed necessary for running prediction-based experiments.

1. Image Recognition Algorithm

In [1]:
# Summary
# The following code imports training images from the Intel Image Classification Dataset, trains a TF model, and then saves a model that can later be used on testing images.

# Intel Dataset Link
# https://www.kaggle.com/datasets/puneet6060/intel-image-classification

# Import Statements
import pandas as pd
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.utils import load_img, img_to_array
from keras.models import load_model


# Constants
# Preset Image Size
img_height = 150
img_width = 150
batch_size = 32

Create TF Model, Import Weights, Compile

In [2]:
# Define Data Generator
df = pd.read_csv('Intel_train_labels.csv')

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
# Load Training Images
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='Intel_seg_train',
    x_col='File Name',
    y_col='Category',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    subset='training'
)
# Define TF Model
model_img_recognition = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Load Pre-Trained Weights for Image Recogntion Model
model_img_recognition.load_weights('TF_Models/ECE50024_ImgRec_TF_Model_V1.h5')

# Define Optimizer w/ Weight Decay
weight_decay = 1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=weight_decay)

# Compile Model
model_img_recognition.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Found 11228 validated image filenames belonging to 6 classes.


2. ACDA Layer Model

In [35]:
# Import necessary libraries
import tensorflow as tf
from keras.layers import Layer
from keras.models import load_model
import os
import glob

In [36]:
# Define Adaptive Convolutional Layer

# Initialize expected input for TF model
inputs = tf.keras.Input(shape=(150, 150, 3))

class AdaptiveConvLayer(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size):
        super(AdaptiveConvLayer, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size

    def build(self, input_shape):
        self.conv_weights = self.add_weight(shape=(self.kernel_size, self.kernel_size, input_shape[-1], self.filters),
                                            initializer='random_normal',
                                            trainable=True)

    def call(self, inputs):
        output = tf.nn.conv2d(inputs, self.conv_weights, strides=[1, 1, 1, 1], padding='SAME')
        return output

# Define the CustomModel class
class CustomModel(tf.keras.Model):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.adaptive_conv = AdaptiveConvLayer(filters=3, kernel_size=3)
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(6, activation='softmax')

    def call(self, inputs):
        x = self.adaptive_conv(inputs)
        x = self.flatten(x)
        x = self.dense(x)
        return x

In [37]:

# Create an instance of the custom model
model_adaptive_conv = CustomModel()
model_adaptive_conv.build((None, 150, 150, 3))

# Output Model Summary
model_adaptive_conv.summary()


Model: "custom_model_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 adaptive_conv_layer_19 (Ada  multiple                 81        
 ptiveConvLayer)                                                 
                                                                 
 flatten_20 (Flatten)        multiple                  0         
                                                                 
 dense_21 (Dense)            multiple                  405006    
                                                                 
Total params: 405,087
Trainable params: 405,087
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Load Pre-Trained Weights for ACDA Model
model_adaptive_conv.load_weights('TF_Models/ECE50024_ACDA_TF_V2.h5')

In [39]:
# Compile Trained Model
model_adaptive_conv.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


3. Import Testing Images and Predict w/ Both Models

In [42]:
# Define Testing Image Directory Path
test_image_dir = 'Intel_seg_pred'

# Creates a list of image paths
img_paths = glob.glob(os.path.join(test_image_dir, '*.jpg'))
images = []
for img_path in img_paths:
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(150, 150))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    #img_array = tf.keras.applications.vgg16.preprocess_input(img_array)
    images.append(img_array)
images = tf.stack(images)

# Use trained models to predict on testing images
adaptive_conv_pred = model_adaptive_conv.predict(images)
img_recognition_pred = model_img_recognition.predict(images)

25/25 [==============================] - 2s 73ms/step


In [43]:
# Output Predictions to Console
print('ACDA Image Recognition Algorithm')
for i, img_path in enumerate(img_paths):
    print('Image:', img_path)
    print('Predictions:', adaptive_conv_pred[i])

print('Initial Image Recognition Algorithm')

for i, img_path in enumerate(img_paths):
    print('Image:', img_path)
    print('Predictions:', img_recognition_pred[i])

ACDA Image Recognition Algorithm
Image: Intel_seg_pred\10060.jpg
Predictions: [0.         0.99999994 0.         0.         0.         0.        ]
Image: Intel_seg_pred\10082.jpg
Predictions: [0.99999994 0.         0.         0.         0.         0.        ]
Image: Intel_seg_pred\10090.jpg
Predictions: [3.3914163e-30 1.5181893e-36 9.1413742e-35 2.5531757e-09 2.4057171e-01
 7.5942820e-01]
Image: Intel_seg_pred\101.jpg
Predictions: [9.9999994e-01 0.0000000e+00 2.5698483e-09 4.3588400e-15 7.9087862e-22
 1.7458536e-22]
Image: Intel_seg_pred\10106.jpg
Predictions: [4.0943581e-24 8.2322434e-38 1.4858159e-14 9.9999994e-01 2.1247641e-10
 0.0000000e+00]
Image: Intel_seg_pred\10158.jpg
Predictions: [0.0000000e+00 0.0000000e+00 0.0000000e+00 9.9999994e-01 1.8231968e-34
 0.0000000e+00]
Image: Intel_seg_pred\10166.jpg
Predictions: [1.4435260e-30 9.9999279e-01 1.3133516e-32 7.1858913e-06 3.3095805e-34
 0.0000000e+00]
Image: Intel_seg_pred\10173.jpg
Predictions: [0.         0.         0.         0.  

4. Class mapping for each model's output, and exporting as csv files.

In [49]:
import csv
image_folder = 'Intel_seg_pred'

# Commented line below to test if it only outputs image id instead of path and .jpg
#files = glob.glob(os.path.join(image_folder, '*.jpg'))
files = os.listdir(image_folder)
class_mapping = ['buildings', 'forest', 'street', 'sea', 'mountain', 'glacier']

# Map Image Recognition Predictions
predicted_classes = [class_mapping[np.argmax(pred)] for pred in img_recognition_pred]

data = list(zip(files, predicted_classes))
# Define Image Recogntion Output File
output_file = 'img_recognition_predictions.csv'

with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File Name', 'Prediction'])
    writer.writerows(data)

# Map ACDA Predictions
predicted_classes = [class_mapping[np.argmax(pred)] for pred in adaptive_conv_pred]

data = list(zip(files, predicted_classes))

# Define ACDA Output File
output_file = 'ACDA_predictions.csv'

with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['File Name', 'Prediction'])
    writer.writerows(data)


Predictions saved to img_recognition_predictions.csv
